In [1]:

import xarray as xr # needed for reading data
import pandas as pd # Used for stroing data
import numpy as np
import pickle as pkl  # Needed for saving model objects
import os
from itertools import repeat # Needed for repeating a variable multiple times

import matplotlib.pyplot as plt

import statsmodels.api as sm    # Used for bot he SARIMA and SARIMAX models
from sklearn import metrics     # Used for importing various performance measures
from multiprocessing import Pool


In [2]:

# Depending on the machine on which the code is run, data might be stored in different directories.
# Indicate which machine is used to make sure the path to the data can be found.
# Can either be "local" or "Snellius"

MACHINE = 'Snellius'

if MACHINE == 'Snellius':
    pred_var_path = '/gpfs/work1/0/ctdas/awoude/Ritten/predictor_vars/' # For retrieving the set of aggregated scaling vectors

    save_dir = pers_file_dir = '/gpfs/work1/0/ctdas/awoude/Ritten/trained_models/' # used for storing the trained model

    results_dir = '/gpfs/work1/0/ctdas/awoude/Ritten/results/' # used for storing the trained model

elif MACHINE == 'local':
    pred_var_path = './' # For retrieving the set of aggregated scaling vectors

    save_dir = pers_file_dir = './models/' # used for storing the trained model

    results_dir = './results/'


In [3]:

def eval_model(model, dat, model_name, test_or_train, show_fit=False):
    '''
    Evaluate the model using the provided testing data
    :param model: The model which is to be tested
    :param dat: The data that is to be used for testing the model. Also includes any predictor variables.
                Can both be used for testing the fit on both trianing data and testing data
    :param model_name: The used ML-algorithm
    :param test_or_train: Flag to indicate whether the passed data is training data ot testing data
    :param show_fit: Flag to indicate whether a plot of the fit should be provided
    :return:
    '''
    flux_dat = dat.prior_flux_per_s.values
    if model_name == "SARIMA":
        true_dat = dat.sf_per_eco.values
        if test_or_train == 'test':
            start_index = len(model.fittedvalues)
            final_model = model.append(true_dat)
        elif test_or_train == 'train':
            start_index = 0
            final_model = model
        else:
            raise Exception(f'test_or_train not specified:{test_or_train}')
        prediction = final_model.get_prediction(start=start_index)
        predict_ci = prediction.conf_int()
        pred_dat = prediction.predicted_mean
        if show_fit:

            # Graph
            fig, ax = plt.subplots(figsize=(9,4))
            title = test_or_train + ' data: predicted sf of ecoregion ' + str(dat.eco_regions.values)
            ax.set(title=title, xlabel='Date', ylabel='Scaling factor')

            # Plot data points
            dat.plot.scatter(x='time',y='sf_per_eco', ax=ax, label='Observed', c='C00')
            # Plot predictions
            plt.plot(dat.time.values, pred_dat, label='One-step-ahead forecast', c='C01')
            ci = predict_ci
            ax.fill_between(dat.time.values, ci[:,0], ci[:,1], color='C01', alpha=0.1)

            legend = ax.legend(loc='lower right')

            plt.show()
    else:
        raise NotImplementedError(f'Model evaluation of {model_name} not implemented')

    # Make sure all provided datasets heve the same length
    assert (len(true_dat)==len(pred_dat)) and (len(true_dat)==len(flux_dat)), 'Passed datasets are do not have the same length'

    # Determine the performance in scaling factor space
    sf_ME = (np.sum(true_dat)-np.sum(pred_dat))/len(true_dat)
    sf_MAE = metrics.mean_absolute_error(true_dat, pred_dat)
    sf_MAPE = metrics.mean_absolute_percentage_error(true_dat, pred_dat)
    sf_RMSE = np.sqrt(metrics.mean_squared_error(true_dat, pred_dat))
    sf_r2 = metrics.r2_score(true_dat, pred_dat)

    # Move evaluation to flux space
    true_flux = true_dat * flux_dat
    pred_flux = pred_dat * flux_dat

    # Determine the performance in flux space
    flux_ME = (np.sum(true_dat)-np.sum(pred_flux))/len(true_dat)
    flux_MAE = metrics.mean_absolute_error(true_flux, pred_flux)
    flux_MAPE = metrics.mean_absolute_percentage_error(true_flux, pred_flux)
    flux_RMSE = np.sqrt(metrics.mean_squared_error(true_flux, pred_flux))
    flux_r2 = metrics.r2_score(true_flux, pred_flux)
    return {'sf_ME_'+test_or_train:sf_ME,
           'sf_MAE_'+test_or_train:sf_MAE,
           'sf_MAPE_'+test_or_train:sf_MAPE,
           'sf_RMSE_'+test_or_train:sf_RMSE,
           'sf_r2_'+test_or_train:sf_r2,
           'flux_ME_'+test_or_train:flux_ME,
           'flux_MAE_'+test_or_train:flux_MAE,
           'flux_MAPE_'+test_or_train:flux_MAPE,
           'flux_RMSE_'+test_or_train:flux_RMSE,
           'flux_r2_'+test_or_train:flux_r2}


def write_model(model, model_name, start_year, eco_region):
    '''
    Function used to save a model in the correct directory with an identifiable name. Uses Pickle for saving the model object
    :param model: The model which is to be saved
    :param model_name: The used ML-algorithm
    :param start_year: The date at which the training data starts
    :param eco_region: The name of the ecoregion to which the model applies
    :return: None
    '''
    file_name = model_name+'_'+str(eco_region)+'.pkl'
    file_dir = save_dir+model_name+'/'+start_year+'/'
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)
    file = file_dir + file_name
    pkl.dump(model, open(file, "wb"))

def train_model(train_dat, model_name, eco_region):
    '''
    Function for training a model on the provided training data
    :param train_dat: The data used for training. Includes both target data and predictor data
    :param model_name: Name of the ML-algorithm to be used for training
    :return: A trianed model
    '''
    if model_name == 'SARIMA':
        target_data = train_dat.sf_per_eco
        model = sm.tsa.statespace.SARIMAX(target_data.values,
                                         order=(2,0,2),             # Defining the regular AR, I and MA dependencies
                                         seasonal_order=(1,0,1,52),     # Defining the seasonal dependencies
                                         trend = 'c'                # Adding an intercept term)
                                          )
        fitted_model=model.fit(maxiter=100) # method='cg'
    else:
        raise NotImplementedError(f'Training of model {model_name} has not been implemented')
#     eco_region = str(train_dat.eco_regions.values)
    start_year = str(train_dat.time.dt.year.min().values)

    # Save model for future usage
    write_model(fitted_model, model_name, start_year, eco_region)
    return fitted_model

def test_eco_region(eco_dat, model_name):
    region, data = eco_dat
    results_df = pd.DataFrame()

    #Set aside the testing data. Using the classical 80%-20% split
    test_ds = data.loc[dict(time=slice("2017", "2020"))]
    region_dat = data.loc[dict(time=slice("2000", "2016"))]
    for year in range(2000, 2017):

        # Determine training data and train model
        train_ds = region_dat.loc[dict(time=slice(str(year), "2016"))]
        trained_model = train_model(train_ds, model_name, region)

        # Evaluate the model, both on training and testing data
        train_results = eval_model(trained_model, train_ds, model_name, 'train', show_fit=False)
        test_results = eval_model(trained_model, test_ds, model_name, 'test', show_fit=False)
        model_params = {
                'eco_region':region,
                'start_year':year,
                'N_train_years':(2017-year),
                'N_train_obs':len(train_ds.time),
                'N_test_years':4,
                'N_test_obs':len(test_ds.time)
        }

        # unpack all dicts to form single results dict
        model_results = {**model_params, **train_results, **test_results}
        if len(results_df) == 0:
            results_df = pd.DataFrame(model_results, index=[0])
        else:
            results_df = results_df.append(model_results, ignore_index=True)
    return results_df

def run_model(model_name, complete_ds):
    eco_region_dat = list(complete_ds.groupby("eco_regions"))
    eco_region_dat = [(region, data.load(scheduler='sync')) for region, data in eco_region_dat]
    with Pool(32) as pool:
        list_of_results = pool.starmap(test_eco_region, zip(eco_region_dat, repeat(model_name)))
    results = pd.concat(list_of_results)
    return results


In [4]:

# Loading all necessary data
with xr.open_dataset(pred_var_path + 'vars_per_eco_update.nc') as ds:
    complete_ds = ds

results = run_model('SARIMA', complete_ds)

results_file = results_dir + 'SARIMA_results.pkl'
print(results)

results.to_pickle(results_file)



/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.14392D+00    |proj g|=  4.53536D-02

At iterate    5    f=  2.13984D+00    |proj g|=  3.60068D-03

At iterate   10    f=  2.13982D+00    |proj g|=  3.08541D-04

At iterate   15    f=  2.13981D+00    |proj g|=  1.05633D-03

At iterate   20    f=  2.13981D+00    |proj g|=  2.85639D-05

At iterate   25    f=  2.13981D+00    |proj g|=  5.42024D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     29     32      1     0     0   

/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.96553D+00    |proj g|=  8.50912D-02

At iterate    5    f=  1.95364D+00    |proj g|=  2.01117D-03

At iterate   10    f=  1.95338D+00    |proj g|=  5.47923D-03

At iterate   15    f=  1.95333D+00    |proj g|=  4.15037D-04

At iterate   20    f=  1.95331D+00    |proj g|=  3.48639D-03

At iterate   25    f=  1.95329D+00    |proj g|=  1.27797D-04

At iterate   30    f=  1.95329D+00    |proj g|=  4.25403D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


...

/tmp/ipykernel_43658/3781297434.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All para

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.13343D+00    |proj g|=  7.24490D-02

At iterate    5    f=  2.12559D+00    |proj g|=  1.13357D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      9     11      1     0     0   4.748D-05   2.126D+00
  F =   2.1255879829581139     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
./models/SARIMA/2015/


/tmp/ipykernel_43658/3781297434.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/user/.pyenv/versions/MasterProject-3.9.2/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All para

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.45712D+00    |proj g|=  4.71729D-02

At iterate    5    f=  2.45079D+00    |proj g|=  1.69062D-03

At iterate   10    f=  2.45051D+00    |proj g|=  9.41503D-03

At iterate   15    f=  2.43861D+00    |proj g|=  3.05342D-02

At iterate   20    f=  2.43403D+00    |proj g|=  2.20849D-02

At iterate   25    f=  2.42899D+00    |proj g|=  8.03576D-03

At iterate   30    f=  2.42353D+00    |proj g|=  4.90617D-02

At iterate   35    f=  2.41851D+00    |proj g|=  7.62160D-02

At iterate   40    f=  2.41269D+00    |proj g|=  9.91614D-02

At iterate   45    f=  2.40803D+00    |proj g|=  1.69257D-01

At iterate   50    f=  2.40220D+00    |proj g|=  1.21649D-01

At iterate   55    f=  2.39590D+00    |proj g|=  2.47784D-01

At iterate   60    f=  2.38767D+00    |proj g|=  1.01321D+00

At iterate   65    f=  2.3


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/tmp/ipykernel_43658/3781297434.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)


In [5]:
pd.read_pickle(results_dir+file_name)

,eco_region,start_year,N_train_years,N_train_obs,N_test_years,N_test_obs,sf_ME_train,sf_MAE_train,sf_MAPE_train,sf_RMSE_train,...,sf_ME_test,sf_MAE_test,sf_MAPE_test,sf_RMSE_test,sf_r2_test,flux_ME_test,flux_MAE_test,flux_MAPE_test,flux_RMSE_test,flux_r2_test
0,1.0,2013.0,4.0,209.0,4.0,208.0,0.001326,0.644575,1.295177,2.056528,...,0.346716,0.580514,2.953288,1.649151,-0.039344,11042.820172,7.781078e+05,2.953288,1.123800e+06,0.735286
1,1.0,2014.0,3.0,157.0,4.0,208.0,-0.017847,0.550743,1.486765,1.692845,...,0.279184,0.588122,3.270719,1.674181,-0.071133,79923.982846,7.332844e+05,3.270719,1.060300e+06,0.764356
2,1.0,2015.0,2.0,105.0,4.0,208.0,-0.015599,0.598733,1.290804,2.015637,...,0.400903,0.613941,2.910919,1.679332,-0.077734,47813.462841,8.190713e+05,2.910919,1.190525e+06,0.702919
3,1.0,2016.0,1.0,53.0,4.0,208.0,-0.192008,0.760977,0.771617,2.813117,...,1.153573,1.808462,5.319124,3.352715,-3.295678,-664725.411662,2.688521e+06,5.319124,4.172654e+06,-2.649416
